<a href="https://colab.research.google.com/github/jeannetous/hello-world/blob/master/INF588_2021_RNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for INF588 2021, RNA alignment folding project

## How to use this document

This Jupyter notebook serves several purposes:

* It introduces the Vienna RNA pacakage for use in our project

* It shows how to set up a development environment, including Python, Biopython and the Vienna RNA package, in Colab. If everything goes well, one can set up this environment by runnning all cells (or by running only the cells of the Installation-Section).

* It provides examples for the use of the Vienna package from Python as well as from command lines - providing a mini-tutorial tailored to our needs.The code is executable on Colab, thanks to the previous point.

* Finally, it provides a pointer to test data and let you upload it to Colab.

One can develop code for the project within Colab, e.g. by extending this document. This should work well as a start and is recommended if you don't have a local setup yet, or if you are using Windows. In the longer run (ideally until the next session), it is recommended to set up a local installation.

Please use the provided information to briefly familiarize yourself with the elementary functionality of the Vienna package before you start coding. Almost ready to use example code is provided and some theory was discussed on the slides, so you should not spend too much time on this.

## The Vienna RNA package

We use the Vienna RNA package (mainly) to predict Minimum Free Energy structures of RNAs as well as to predict base pair probabilities; moreover, to call RNAalifold for comparison and benchmarks. These uses will be demonstrated in this document.

The Vienna package is available from

https://www.tbi.univie.ac.at/RNA/

At this page one finds (among other things)

* **Download links and installation instructions.** However note that for work in the Colab environment, it is recommended to follow the instructions / run the code below. For local installation, Conda is recommended for Mac/Linux. For **Windows**: use the Windows installer of ViennaRNA (which installs command line tools, but no Python bindings) or work in the Colab environment (such that the package functions can be more easily used from Python). Note that one can still install Python and Biopython via Conda under Windows. 

* **Documentation of command line tools and Python bindings** (https://www.tbi.univie.ac.at/RNA/documentation.html). Most essential information to use the package for our purposes will be given below.



## Installation of Biopython and module RNA from the Vienna RNA package in Colab

Before we start, let's do a quick check, whether Biopython and/or RNA is already available. 

In [ ]:
def check(name,check):
    print("Check",name,"\t... ",end="")
    try:
        check(); print("seems available")
    except: print("not (properly) installed")

def check_bp():
    from Bio.Seq import Seq; Seq('acgtgact')
def check_vrna():
    import RNA; RNA.fold_compound('acgcuacg').mfe()

check("Biopython",check_bp)
check("Vienna RNA",check_vrna)

For installing Biopython and the Vienna RNA package, we will first enable package management by Conda, such that we can then install the package by a single command: ```conda install bioconda viennarna```. How to make Conda available on Colab is taken from https://towardsdatascience.com/conda-google-colab-75f7c867a522. Note that this is slightly more complex than a local installation on Linux (or Mac), which could similarily work by first installing conda (from miniconda) and then running the above install command (see also the Section on local installation in the Biopython notebook:
https://colab.research.google.com/drive/1CvjJCVFMHfdlRnICuTYMYH8pSHtEr2rW?usp=sharing).

We start by running a view checks to ensure that the installation still fits our collab environment.

In [ ]:
!which python # should return /usr/local/bin/python
!python --version  # we assume Python 3.6.9
!echo $PYTHONPATH  # whether this is set or not,
%env PYTHONPATH=   #   we will just unset it

We are installing the Conda distribution miniconda. This will work if the above checks returned the expected results. Otherwise, the procedure may need some update. 

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

Check whether this worked as intended


In [ ]:
!which conda # should return /usr/local/bin/conda
!conda --version # should return 4.5.4
!which python # still returns /usr/local/bin/python
!python --version # now returns Python 3.6.5 :: Anaconda, Inc.

If we got the expected results, we furthermore update conda  (this can take a while) and then check again, whether the update succeeded.

In [ ]:
%%bash

conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

The checks should report at least the same version numbers of Conda and Python (otherwise, it is very likely that something went wrong before).

In [ ]:
!conda --version # now returns 4.8.3
!python --version # now returns Python 3.6.12 :: Anaconda, Inc.

In [ ]:
import sys
sys.path.append("/usr/local/lib/python3.6/site-packages")

From here, the Conda package manager is set up and ready to use. Let's install the Vienna RNA Pacakge and it's module RNA (which was our motivation to install conda in the first place):

In [ ]:
!conda install -y -c bioconda viennarna biopython

It remains to check the installation, before we go on:


In [ ]:
try:
  from Bio.Seq import Seq
  import RNA
  seq = Seq('gcatgacgttattacgactctgtcacgccgcggtgcgactgaggcgtggcgtctgctggg')
  fc = RNA.fold_compound(str(seq))
  fc.mfe()
  print("So far, so good :)")
except:
  print("Something went wrong")

## Usage of the Vienna RNA package

### Using Python bindings / module RNA

One finds more than the below purely elementary usage in the API documentation:

https://www.tbi.univie.ac.at/RNA/documentation.html#api

#### Predicting the Minimum free energy structure

In [ ]:
## Code adapted from the Vienna RNA package API documentation

import RNA
# The RNA sequence
sequence = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"

# create a new model details structure
md = RNA.md()

# optionally one could change all kinds of parameters
# md.temperature = 25.0 # 25 Deg Celcius
# md.dangles = 1 # keep default dangles=2 for compatibility with partition folding

# create a fold compound
fc = RNA.fold_compound(sequence, md)

# predict Minmum Free Energy and corresponding secondary structure
(ss, mfe) = fc.mfe()

# print sequence, structure and MFE
print("%s\n%s [ %6.2f ]\n" % (sequence, ss, mfe))

# parse the dot-bracket structure:
# RNA.ptable returns a table, such that table[i]=j if (i.j) pair or 0 if i is unpaired.
# note: positions are 1-based; table[0] contains the length of the structure.
pt = RNA.ptable(ss)
print(pt)

#### Predicting partition funtions and base pair probabilities

In [ ]:
## Code adapted from the Vienna RNA package API documentation

import RNA
sequence = "GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA"
# create model details
md = RNA.md()
# activate unique multibranch loop decomposition
md.uniq_ML = 1
# create fold compound object
fc = RNA.fold_compound(sequence, md)
# compute MFE
(ss, mfe) = fc.mfe()
# rescale Boltzmann factors according to MFE; rescaling avoids numerical problems for long sequences
fc.exp_params_rescale(mfe)
# compute partition function to fill DP matrices
fc.pf()

# get a matrix of base pair probabilities (attention: this matrix is 1-based), only entries i<j are meaningful
bpp = fc.bpp()
print(bpp)

# compute unpaired probabilities (0-based):
unp = [1-sum(bpp[i+1]) for i in range(0,len(sequence))]
print(unp)

# produce simple 'dot plot'
import matplotlib.pyplot as plt
bpp2 = [p**2 for row in bpp[1:] for p in row[1:]]
plt.imshow(bpp)


#### Predicting RNAalifold structures

In [ ]:
import RNA
# The RNA sequence

alignment = ["GGAGGAUUAGCUCAGCUGGGAGAGCAUCUGCCUUACAAGCAGAGGG-----------UCGGCGGUUCGAGCCCGUCAUCCUCC",
"GCCUUCCUAGCUCAG-UGGUAGAGCGCACGGCUUUUAACCGUGUGG-----------UCGUGGGUUCGAUCCCCACGGAAGGC",
"GCCUUUAUAGCUUAG-UGGUAAAGCGAUAAACUGAAGAUUUAUUUA-----------CAUGUAGUUCGAUUCUCAUUAAGGGC",
"GCGGAUAUAACUUAGGGGUUAAAGUUGCAGAUUGUGGCUCUGAAAA------------CACGGGUUCGAAUCCCGUUAUUCGC",
"GGAAAAUU-GAUCAUCGGCAAGAUAAGUUAUUUACUAAAUAAUAGGAUUUAAUAACCUGGUGAGUUCGAAUCUCACAUUUUCC"
]

# create a new model details structure
md = RNA.md()
# optionally one could change some parameters
# md.temperature = 25.0 # 25 Deg Celcius
# md.dangles = 1 # keep default 2 for compatibility with partition folding
# create a fold compound
fc = RNA.fold_compound(alignment, md)
# predict Minmum Free Energy and corresponding secondary structure
(ss, mfe) = fc.mfe()
conservation_score = fc.eval_covar_structure(ss)
print("%s\n%s [ %6.2f, %6.2f ]\n" % ('\n'.join(alignment), ss, mfe, conservation_score))

### Using command line tools

We demonstrate very basic usage (still sufficient for our purposes), please find more information in the online documentation, man pages or using option --help of the tools. 

#### MFE structure prediction

In [ ]:
%%bash
echo GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA | RNAfold 

Additionally, a 2D plot of the predicted structure is written to rna.ps and can be viewe using a postscript / pdf viewer.

#### Partition function and base pair probability prediction

In [ ]:
%%bash
echo GAGUAGUGGAACCAGGCUAUGUUUGUGACUCGCAGACUAACA | RNAfold -p

Additionally to rna.ps this command writes dot.ps; the latter file is viewable and readable/parsable. It contains the base pair probabilities.

One can extract these probabilities, e.g. as follows

In [ ]:
import re
import math
with open('dot.ps') as fh:
    bplist = list()
    for line in fh:
        m = re.match(r"(\d+) (\d+) (\d+.\d+) ubox$",line)
        if m:
            i,j,p2 = int(m[1]),int(m[2]),float(m[3])
            p = math.sqrt(p2)
            bplist.append((i,j,p))
# bplist is a list of base pairs with probabilities; only probabilities >= 10^6.
print(bplist)

#### Folding an alignment

In [ ]:
%%bash

# first generate an input file
echo >alignment.aln "CLUSTAL W
AF008220           GGAGGAUUAGCUCAGCUGGGAGAGCAUCUGCCUUACAAGCAGAGGG-----------UCGGCGGUUCGAGCCCGUCAUCCUCC
Z11880             GCCUUCCUAGCUCAG-UGGUAGAGCGCACGGCUUUUAACCGUGUGG-----------UCGUGGGUUCGAUCCCCACGGAAGGC
X02172             GCCUUUAUAGCUUAG-UGGUAAAGCGAUAAACUGAAGAUUUAUUUA-----------CAUGUAGUUCGAUUCUCAUUAAGGGC
M68929             GCGGAUAUAACUUAGGGGUUAAAGUUGCAGAUUGUGGCUCUGAAAA------------CACGGGUUCGAAUCCCGUUAUUCGC
D10744             GGAAAAUU-GAUCAUCGGCAAGAUAAGUUAUUUACUAAAUAAUAGGAUUUAAUAACCUGGUGAGUUCGAAUCUCACAUUUUCC
"

# call RNAalifold on the file
RNAalifold --aln < alignment.aln

This writes additional files alirna.ps and aln.ps.

## Test data

Find test data here
https://www.lix.polytechnique.fr/~will/Teaching/INF588-2021/Testdata

Can your program handle all of the diverse examples (in different file formats)?

Here is one way to download the data to Colab by downloading my archive and unpacking. The files will be written to subdirectory Testdata

In [ ]:
%%bash
ARC=testdata.tar.gz
if [ -e "$ARC" ] ; then rm -rf "$ARC" ; fi
wget "https://www.lix.polytechnique.fr/~will/Teaching/INF588-2021/$ARC" > /dev/null 2>&1
tar xvzf "$ARC"

## You can start coding here...

Some words about coding environments (as already discussed before): It can be convenient to start the work for the project on Colab - in this way, one does not have to install Python, Bioconda and Vienna RNA locally. In the longer run (from next session), it will have advantages to run in a local Jupyter Notebook (e.g. install via ```conda install jupyter```) or some other local Python environment. This is mainly, since running larger computations in Colab is tricky and as well, Colab will require you to reinstall conda, biopython and viennarna in each new Colab session.

Unfortunately, Windows-users have the problem that the Vienna RNA package Python bindings / using ```module RNA``` will not work locally - a reasonable workaround can be to call the RNA prediction tools via system calls and parse their output (the Section "Using command line tools" shows how to do this).
